In [ ]:
import pandas as pd


# --- Step 1: Load All Your Datasets ---
try:
    router_a_df = pd.read_csv('data/raw/Router_A_router_log_15_days.csv')
    router_b_df = pd.read_csv('data/raw/Router_B_router_log_15_days.csv')
    router_c_df = pd.read_csv('data/raw/Router_C_router_log_15_days.csv')
    app_usage_df = pd.read_csv('data/raw/application_usage.csv')
    user_activity_df = pd.read_csv('data/raw/user_activity.csv')
    external_factors_df = pd.read_csv('data/raw/external_factors.csv')
    config_history_df = pd.read_csv('configuration_history.csv') # Load the new file
except FileNotFoundError:
    print("Error: Make sure all CSV files are uploaded to your Colab session.")
    exit()

# --- Step 2: Combine Router Logs and Create the New Congestion Flag ---
all_routers_df = pd.concat([router_a_df, router_b_df, router_c_df], ignore_index=True)

def create_new_flag(row):
    if row['Bandwidth Allocated (MB/s)'] == 0:
        return 0
    utilization = row['Bandwidth Used (MB/s)'] / row['Bandwidth Allocated (MB/s)']
    latency = row['Latency (ms)']
    if utilization > 0.85 or (utilization > 0.7 and latency > 45):
        return 1
    return 0

all_routers_df['New_Flag'] = all_routers_df.apply(create_new_flag, axis=1)
all_routers_df['Timestamp'] = pd.to_datetime(all_routers_df['Timestamp'])
all_routers_df['Date'] = pd.to_datetime(all_routers_df['Timestamp'].dt.date)

# --- Step 3: Aggregate Daily Datasets ---
app_usage_df['Date'] = pd.to_datetime(app_usage_df['Date'])
user_activity_df['Date'] = pd.to_datetime(user_activity_df['Date'])
external_factors_df['Date'] = pd.to_datetime(external_factors_df['Date'])
config_history_df['Date'] = pd.to_datetime(config_history_df['Date'])

daily_app_summary = app_usage_df.groupby('Date').agg(total_peak_app_traffic=('Peak_Traffic_MB', 'sum')).reset_index()
daily_user_summary = user_activity_df.groupby('Date').agg(total_logins=('Login_Count', 'sum')).reset_index()

# --- NEW: Process Configuration History ---
# Rename 'Device' to 'Device Name' for a consistent merge key
config_history_df.rename(columns={'Device': 'Device Name'}, inplace=True)
# Count the number of changes per device per day
daily_config_summary = config_history_df.groupby(['Date', 'Device Name']).size().reset_index(name='Num_Config_Changes')

# --- Step 4: Merge All Data into a Single DataFrame ---
# Merge the daily summaries first
merged_df = pd.merge(all_routers_df, daily_app_summary, on='Date', how='left')
merged_df = pd.merge(merged_df, daily_user_summary, on='Date', how='left')
merged_df = pd.merge(merged_df, external_factors_df, on='Date', how='left')

# Now merge the configuration history using the composite key
merged_df = pd.merge(merged_df, daily_config_summary, on=['Date', 'Device Name'], how='left')

# Fill any missing values after the merges
merged_df.fillna({'Event': 'None', 'Impact': 'None', 'Num_Config_Changes': 0}, inplace=True)
# Convert Num_Config_Changes to integer
merged_df['Num_Config_Changes'] = merged_df['Num_Config_Changes'].astype(int)

# --- Step 5: Sort the DataFrame ---
sorted_df = merged_df.sort_values(by=['Timestamp', 'Device Name']).reset_index(drop=True)

# --- Step 6: Save and Download the Final File ---
file_name = 'final_model_ready_dataset.csv'
sorted_df.to_csv(file_name, index=False)

print(f"Successfully created the final file '{file_name}' with all data merged.")
print("\nHere are the first 10 rows to show the final result, including 'Num_Config_Changes':")
print(sorted_df[['Timestamp', 'Device Name', 'New_Flag', 'total_logins', 'Num_Config_Changes']].head(10))

print("\nStarting download...")



Error: Make sure all CSV files are uploaded to your Colab session.


NameError: name 'config_history_df' is not defined

: 